In [19]:
import elasticc_for_etienne as efe
from typing import Dict, List, Union
import glob
from itertools import chain
from astropy.table import Table
import pickle

In [68]:
bazin_cut = True

name = 'ELASTICC_TRAIN_uLens-Binary'

path = f'/media/ELAsTICC/data/training_sample/{name}'
heads = sorted(glob.glob(f'{path}/*_HEAD.FITS.gz'))
phots = sorted(glob.glob(f'{path}/*_PHOT.FITS.gz'))
assert len(heads) != 0, 'no *_HEAD_FITS.gz are found'
assert len(heads) == len(phots), 'there are different number of HEAD and PHOT files'

if bazin_cut:
    min_det_per_band = {b'g ': 4, b'r ': 4, b'i ': 4}
    min_total_det = 12
    
else:
    min_det_per_band = {b'g ': 2, b'r ': 2, b'i ': 2}
    min_total_det = 10
    
has_peak = False

# merge list of lists into a single list
lcs = list(chain.from_iterable(
    efe.parse_fits_snana(head, phot, min_det_per_band=min_det_per_band, min_total_det=min_total_det, has_peak=has_peak)
                     for head, phot in zip(heads, phots)
))

In [69]:
# Normallize values
for idx, lc in enumerate(lcs):
    
    gband = lc[lc['BAND']=='g ']
    
    maxi = gband['FLUXCAL'].max()
    lc['max_flux'] = maxi
    t_maxi = gband['MJD'][gband['FLUXCAL'].argmax()]
    lc['max_flux_time'] = t_maxi
    
    lc['FLUXCAL'] = lc['FLUXCAL']/maxi
    lc['FLUXCALERR'] = lc['FLUXCALERR']/maxi
    
    lc['MJD'] = lc['MJD']-t_maxi

In [70]:
if bazin_cut:
    file = f'preprocess_data/{name}_bazin_cut.pkl'
    
else:
    file = f'preprocess_data/{name}.pkl'

with open(file, 'wb') as handle:
    pickle.dump(lcs, handle)

In [71]:
print('OK')

OK


In [72]:
len(lcs)

3115